In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../../")

In [ ]:
import tensorflow as tf

from choice_learn.datasets import load_swissmetro
from choice_learn.models.base_model import ChoiceModel

In [ ]:
class ExampleCustomizedModel(ChoiceModel):
    def __init__(self, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.n_neurons = n_neurons

        # Items Features Layer
        self.dense_items = tf.keras.layers.Dense(units=n_neurons, activation="sigmoid")

        # Shared Features Layer
        self.dense_shared = tf.keras.layers.Dense(units=n_neurons, activation="sigmoid")

        # Third layer: embeddings to utility (dense representation of features > U)
        self.final_layer = tf.keras.layers.Dense(units=1, activation="linear")

    @property
    def trainable_weights(self):
        """Access model's trainable_weights.

        Returns
        -------
        list
            list of trainable_weights
        """
        return self.dense_items.trainable_variables\
              + self.dense_shared.trainable_variables\
                  + self.final_layer.trainable_variables

    def compute_batch_utility(self,
                              shared_features_by_choice,
                              items_features_by_choice,
                              available_items_by_choice,
                              choices):
        """Compute batch utility from features."""
        _, _ = available_items_by_choice, choices
        # We apply the neural network to all items_features_by_choice for all the items
        # We then concatenate the utilities of each item of shape (n_choices, 1) into
        # a single one of shape (n_choices, n_items)
        shared_embeddings = self.dense_shared(shared_features_by_choice)

        # Iterate over items
        items_features_embeddings = []
        for i in range(items_features_by_choice.shape[1]):
            # Utility is Dense(embeddings sum)
            item_embedding = self.dense_items(items_features_by_choice[:, i])
            item_embedding = tf.keras.activations.sigmoid(shared_embeddings + item_embedding)
            items_features_embeddings.append(self.final_layer(item_embedding))

        # Concatenation to get right shape (n_choices, n_items, )
        return tf.concat(items_features_embeddings, axis=1)

In [ ]:
dataset = load_swissmetro()

In [ ]:
model = ExampleCustomizedModel(n_neurons=10, optimizer="Adam", batch_size=128)
history = model.fit(dataset)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(history["train_loss"])

In [ ]:
print(model.evaluate(dataset))